In [1]:
import random
from datetime import datetime, date, timedelta
from itertools import chain
import pandas as pd
import networkx as nx
from itertools import combinations  
import numpy as np
import time

class CareTeam:
    def __init__(self, note_df, discharge_id, care_team):
        self.note_df = note_df  
        self.discharge_id = discharge_id
        self.care_team = care_team
        self.care_team_edges = [sorted(edge) for edge in list(combinations(care_team, 2))]
        self.G = nx.Graph()
        self.unique_dates = note_df.date.unique()
        self.__create_graph()
    
    def __create_graph(self):
        for note_date in self.unique_dates:
            notes_for_date = self.note_df.query('date == @note_date')
            discharge_ids_for_date = notes_for_date.discharge_id.unique()
            for discharge_id in discharge_ids_for_date:
                drs_for_discharge_id = notes_for_date.query('discharge_id == @discharge_id').dr.unique()
                care_team_edges_for_discharge_id = [edge for edge in list(combinations(drs_for_discharge_id, 2)) 
                                                    if sorted(edge) in self.care_team_edges]
                for edge in care_team_edges_for_discharge_id: 
                    self.__add_edge_to_G(edge)
        
    def __add_edge_to_G(self, edge):
        data = self.G.get_edge_data(*edge, default=None)
        weight = 1 if data is None else data['weight'] + 1
        self.G.add_edge(*edge, weight=weight)

class CareDate:
    def __init__(self, note_df, notes_for_care_date, date_of_care):
        self.note_df = note_df
        self.notes_for_care_date = notes_for_care_date
        self.date_of_care = date_of_care
        self.care_team_dict = {}
        self.__create_care_team_dict()
    
    def __create_care_team_dict(self):
        discharge_ids_for_date = self.notes_for_care_date.discharge_id.unique()
        for discharge_id in discharge_ids_for_date:
            drs_for_discharge_id = self.notes_for_care_date.query('discharge_id == @discharge_id').dr.unique()
            self.care_team_dict[discharge_id] = drs_for_discharge_id
        
    def __iter__(self):
        for discharge_id in self.care_team_dict:
            care_team = self.care_team_dict[discharge_id]  
            yield CareTeam(self.note_df, discharge_id, care_team)
    
class TeamworkStudy:
    def __init__(self, note_df, window_in_days, step_in_days):
        note_df.sort_values('date', inplace=True)
        self.note_df = note_df
        self.DELTA = np.timedelta64(window_in_days, 'D')
        step = np.timedelta64(step_in_days, 'D')
        first_date = note_df['date'].iloc[0] 
        last_date = note_df['date'].iloc[-1]
        self.date_range = np.arange(first_date, last_date - self.DELTA, step)
        
    def __iter__(self):
        for start_date in self.date_range:
            end_date = start_date + self.DELTA
            date_of_care = end_date + np.timedelta64(1, 'D')
            notes_in_window = self.note_df.query('date >= @start_date & date <= @end_date')
            notes_for_care_date = self.note_df.query('date == @date_of_care')
            num_rows = len(notes_for_care_date.index)
            if num_rows == 0: continue
            yield CareDate(notes_in_window, notes_for_care_date, date_of_care)
            

In [3]:
note_df = pd.read_csv('../data/notes_test.csv', parse_dates=[2])

WINDOW = 90
STEP = 1

get_care_dates = TeamworkStudy(note_df, WINDOW, STEP)
    
for care_date in get_care_dates:
    print(f'Care Date: {care_date.date_of_care}')
    for care_team in care_date:
        weights = nx.get_edge_attributes(care_team.G, 'weight')
        print(f'Discharge id: {care_team.discharge_id}')
        print(weights)

Care Date: 2019-04-15T00:00:00.000000
Discharge id: 6
{('Albert Romero', 'Margie Meyer'): 2, ('Albert Romero', 'Evan Frazier'): 2, ('Albert Romero', 'Myrtle George'): 2, ('Albert Romero', 'Victoria Washington'): 1, ('Albert Romero', 'Brad Palmer'): 1, ('Albert Romero', 'Neil Mitchell'): 1, ('Margie Meyer', 'Evan Frazier'): 2, ('Margie Meyer', 'Myrtle George'): 2, ('Margie Meyer', 'Victoria Washington'): 1, ('Margie Meyer', 'Brad Palmer'): 1, ('Margie Meyer', 'Neil Mitchell'): 1, ('Evan Frazier', 'Myrtle George'): 2, ('Evan Frazier', 'Victoria Washington'): 1, ('Evan Frazier', 'Brad Palmer'): 1, ('Evan Frazier', 'Neil Mitchell'): 1, ('Myrtle George', 'Victoria Washington'): 1, ('Myrtle George', 'Brad Palmer'): 1, ('Myrtle George', 'Neil Mitchell'): 1, ('Victoria Washington', 'Brad Palmer'): 1, ('Victoria Washington', 'Neil Mitchell'): 1, ('Brad Palmer', 'Neil Mitchell'): 1}
Care Date: 2019-08-15T00:00:00.000000
Discharge id: 7
{}
